## Subgraph classification 

In [2]:
#Imports
import numpy as np
from NNetwork import NNetwork as nn
import networkx as nx
#import utils.NNetwork as nn
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import metrics, model_selection
from tqdm import trange
from sklearn.cluster import KMeans
import matplotlib.gridspec as gridspec
from tqdm import trange
from numpy import genfromtxt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
def compute_accuracy_metrics(Y_test, P_pred, use_opt_threshold=False, verbose=False):
    # y_test = binary label 
    # P_pred = predicted probability for y_test
    # compuate various binary classification accuracy metrics
    fpr, tpr, thresholds = metrics.roc_curve(Y_test, P_pred, pos_label=None)
    mythre = thresholds[np.argmax(tpr - fpr)]
    myauc = metrics.auc(fpr, tpr)
    # print('!!! auc', myauc)
    
    # Compute classification statistics
    threshold = 0.5
    if use_opt_threshold:
        threshold = mythre
    
    Y_pred = P_pred.copy()
    Y_pred[Y_pred < threshold] = 0
    Y_pred[Y_pred >= threshold] = 1

    mcm = confusion_matrix(Y_test, Y_pred)
    tn = mcm[0, 0]
    tp = mcm[1, 1]
    fn = mcm[1, 0]
    fp = mcm[0, 1]

    accuracy = (tp + tn) / (tp + tn + fp + fn)
    sensitivity = tn / (tn + fp)
    specificity = tp / (tp + fn)
    precision = tp / (tp + fp)
    fall_out = fp / (fp + tn)
    miss_rate = fn / (fn + tp)

    # Save results
    results_dict = {}
    results_dict.update({'Y_test': Y_test})
    results_dict.update({'Y_pred': Y_pred})
    results_dict.update({'AUC': myauc})
    results_dict.update({'Opt_threshold': mythre})
    results_dict.update({'Accuracy': accuracy})
    results_dict.update({'Sensitivity': sensitivity})
    results_dict.update({'Specificity': specificity})
    results_dict.update({'Precision': precision})
    results_dict.update({'Fall_out': fall_out})
    results_dict.update({'Miss_rate': miss_rate})
    
    if verbose:
        for key in [key for key in results_dict.keys()]:
            print('% s ===> %.3f' % (key, results_dict.get(key)))
    return results_dict

In [4]:
# X: a list of NNetwork objects
# num_edges, num_nodes, min_degree, max_degree, diameter
import math

def MACC_nx(G, k):
    # G = networkx Network
    # k = length of chain motif
    # Compute k x k Matrix of Average Clustering Coefficients
    G_nn = nn.NNetwork()
    G_nn.add_edges( list(G.edges) )
    X0, embs0 = G_nn.get_patches(k=k, sample_size=1000, skip_folded_hom=False, sampling_alg = 'pivot')
    MACC = np.sum(X0, axis=1)/X0.shape[1]
    MACC = MACC.reshape(k,k)
    return MACC

def datagen_graph_features(X, k0_list):
    # generate the toy dataset
    # file = open(file_name, 'w+', newline='')
    output_dict_list = []
    
    header = ["num_edges", "num_nodes", "min_degree", "max_degree", "diameter", 
              "degree_assortativity_coef", "num_clique", "avg_clustering_coef", 
              "density"]
  
    for i in trange(len(X)): #X.shape[1]: total number of graphs
                                #X.shape[0]: num_nodes^2
        output_dict = {}
        G_nn = X[i]
        G=nx.Graph(G_nn.get_edges())
        
        num_edges = G.number_of_edges()
        min_degree = min(list(G.degree), key=lambda x: x[1])[1]
        max_degree = max(list(G.degree), key=lambda x: x[1])[1]
        diameter = nx.diameter(G)

        degree_assortativity_coef = nx.degree_assortativity_coefficient(G)
        num_clique = nx.graph_clique_number(G)
        avg_clustering_coef = nx.average_clustering(G)
        #small_world_coef = nx.omega(G)
        density = nx.density(G)

        output_dict.update({"num_nodes": len(G.nodes())})
        output_dict.update({"num_edges": G.number_of_edges()})
        output_dict.update({"min_degree": min(list(G.degree), key=lambda x: x[1])[1]})
        output_dict.update({"max_degree": max(list(G.degree), key=lambda x: x[1])[1]})
        output_dict.update({"diameter": nx.diameter(G)})
        a = nx.degree_assortativity_coefficient(G)
        if math.isnan(a):
            a = 0
        output_dict.update({"degree_assortativity_coef": a})
        output_dict.update({"num_cliques": nx.graph_clique_number(G)})
        output_dict.update({"Avg_clustering_coeff": nx.average_clustering(G)})
        output_dict.update({"edeg_density": nx.density(G)})
        for k0 in k0_list:
            output_dict.update({"MACC k0={}".format(k0): MACC_nx(G, k=k0)})

        output_dict_list.append(output_dict)
        
    return output_dict_list

def run_binary_classification(output_dict_list, y, scale=1):

    X_train_idx, X_test_idx, y_train, y_test = train_test_split(np.arange(len(y)), y, test_size=0.50, random_state=42)        

    
    keys = list(output_dict_list[0].keys())
    results_dict_total = {}
    for key in keys:
        feature_vector_list = []
        for i in np.arange(len(output_dict_list) ):
            output_dict = output_dict_list[i]
            key
            if key.split(" ")[0] == "MACC":
                feature_vector_list.append(output_dict.get(key).reshape(-1,1)) 
            else: 
                feature_vector_list.append(output_dict.get(key)) 

        X = np.asarray(feature_vector_list)
        if key.split(" ")[0] == "MACC":
            X = X.reshape(len(feature_vector_list), -1) 
        else: 
            X = X[:,np.newaxis]

        print("X.shape", X.shape)

        X_train = X[X_train_idx,:]
        X_test = X[X_test_idx,:]
    
        scale0 = 1
        if key.split(" ")[0] == "MACC":
            scale0 = scale
        clf = LogisticRegression(random_state=0).fit((X_train-np.mean(X_train, axis=0))/scale0, y_train)
        y_pred = clf.predict((X_test-np.mean(X_test, axis=0))/scale0)
        P_pred = clf.predict_proba((X_test-np.mean(X_test, axis=0))/scale0)

        results_dict = compute_accuracy_metrics(y_test, P_pred[:,1], use_opt_threshold=False, verbose=False)

        print("method = {}, AUC = {:f}".format(key, results_dict.get("AUC")))

        results_dict_total.update({key: results_dict})
    return results_dict_total
    

In [6]:
path = "Output_files1/subgraph_classification_data30_0.npy"
results_0 = np.load(path, allow_pickle=True).item()

In [7]:
results_0.keys()

dict_keys(['Caltech36-Simmons81', 'Caltech36-Reed98', 'Caltech36-NYU9', 'Caltech36-Virginia63', 'Caltech36-UCLA26', 'Caltech36-Wisconsin87', 'Simmons81-Reed98', 'Simmons81-NYU9', 'Simmons81-Virginia63', 'Simmons81-UCLA26', 'Simmons81-Wisconsin87', 'Reed98-NYU9', 'Reed98-Virginia63', 'Reed98-UCLA26', 'Reed98-Wisconsin87', 'NYU9-Virginia63', 'NYU9-UCLA26', 'NYU9-Wisconsin87', 'Virginia63-UCLA26', 'Virginia63-Wisconsin87', 'UCLA26-Wisconsin87'])

In [8]:
pairs_list = list(results_0.keys())
len(pairs_list)

21

In [9]:
method_list_new = [" ".join(method.split("_")) for method in method_list]
method_list_new

['edeg density',
 'min degree',
 'max degree',
 'diameter',
 'degree assortativity coef',
 'num cliques',
 'Avg clustering coeff',
 'MACC k0=5',
 'MACC k0=10',
 'MACC k0=15',
 'MACC k0=20']

In [11]:
import pandas as pd

method_list = ['edge_density', 'min_degree', 'max_degree', 'diameter', 'degree_assortativity_coef', 'num_cliques', 'Avg_clustering_coeff',  'MACC k0=5', 'MACC k0=10', 'MACC k0=20', 'MACC k0=30']
pairs_list = list(results_0.keys())
method_list_new = [" ".join(method.split("_")) for method in method_list]

method_0 = []
full_result_list = []

for i in trange(10):
    path = "Output_files1/subgraph_classification_data30_"+str(i)+".npy"
    results_0 = np.load(path, allow_pickle=True).item()

    AUC_list_total = []
    for method in method_list:
        AUC_list = []
        for idx in range(len(pairs_list)):
            #print("method={}, pair={}".format(method,pairs_list[idx] ))

            pair = pairs_list[idx]
            result1 = results_0.get(pair)
            result11 = result1.get("k=30")
            #print(result11)
            #print("result11.get(method)", result11.get(method).keys())
            #print("method={}, pair={}".format(method, pair))
            AUC = result11.get(method).get("AUC")
            AUC_list.append(np.round(AUC, 3))

            
        AUC_list_total.append(AUC_list)
        
    full_result_list.append(AUC_list_total)
    #full_result_list.append([AUC_list_total, std_list_total])


A = np.asarray(full_result_list)

100%|███████████████████████████████████████████| 10/10 [00:43<00:00,  4.35s/it]


In [12]:
A = np.asarray(full_result_list)

AUC_array = np.round(np.mean(A, axis=0),3).T
std_array = np.round(np.std(A, axis=0),3).T
     
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 10)
# save mean 
result = pd.DataFrame(data=AUC_array)
result = result.set_axis(method_list_new, axis=1, inplace=False)
#result.insert(0, column='xi', value= xi_0)
#result.insert(0, column='Method', value= method_0)
#result = round(result.groupby(['Method', 'xi'], sort=False).agg(['mean','std']),3)
result["Networks"] = pairs_list
result = result.set_index("Networks")
result
result.to_csv('Figures/table.csv', index=True)

# save std
result_std = pd.DataFrame(data=std_array)
result_std = result_std.set_axis(method_list_new, axis=1, inplace=False)
#result.insert(0, column='xi', value= xi_0)
#result.insert(0, column='Method', value= method_0)
#result = round(result.groupby(['Method', 'xi'], sort=False).agg(['mean','std']),3)
result_std["Networks"] = pairs_list
result_std = result_std.set_index("Networks")
result_std.to_csv('Figures/table_std.csv', index=True)

In [13]:
result

,edge density,min degree,max degree,diameter,degree assortativity coef,num cliques,Avg clustering coeff,MACC k0=5,MACC k0=10,MACC k0=20,MACC k0=30
Networks,,,,,,,,,,,
Caltech36-Simmons81,0.853,0.635,0.758,0.846,0.640,0.729,0.751,0.897,0.932,0.936,0.936
Caltech36-Reed98,0.876,0.615,0.792,0.835,0.601,0.749,0.784,0.910,0.935,0.936,0.933
Caltech36-NYU9,0.867,0.592,0.778,0.793,0.620,0.747,0.783,0.896,0.919,0.921,0.921
Caltech36-Virginia63,0.876,0.603,0.793,0.819,0.650,0.764,0.782,0.890,0.921,0.926,0.918
Caltech36-UCLA26,0.900,0.564,0.822,0.852,0.652,0.762,0.788,0.919,0.933,0.935,0.935
Caltech36-Wisconsin87,0.858,0.602,0.762,0.822,0.622,0.735,0.772,0.879,0.921,0.920,0.923
Simmons81-Reed98,0.836,0.597,0.761,0.796,0.594,0.734,0.756,0.874,0.918,0.920,0.916
Simmons81-NYU9,0.857,0.588,0.759,0.837,0.634,0.739,0.758,0.901,0.929,0.936,0.932
Simmons81-Virginia63,0.869,0.611,0.784,0.832,0.628,0.757,0.783,0.902,0.925,0.929,0.929


In [92]:
result_std

,edeg density,min degree,max degree,diameter,degree assortativity coef,num cliques,Avg clustering coeff,MACC k0=5,MACC k0=10,MACC k0=15,MACC k0=20
Networks,,,,,,,,,,,
Caltech36-Simmons81,0.037,0.029,0.022,0.043,0.068,0.040,0.044,0.040,0.027,0.026,0.020
Caltech36-Reed98,0.035,0.054,0.040,0.055,0.033,0.027,0.042,0.038,0.023,0.021,0.025
Caltech36-NYU9,0.034,0.046,0.035,0.036,0.067,0.039,0.034,0.023,0.021,0.025,0.017
Caltech36-Virginia63,0.035,0.041,0.044,0.033,0.112,0.032,0.059,0.031,0.017,0.021,0.019
Caltech36-UCLA26,0.033,0.032,0.042,0.022,0.031,0.046,0.049,0.021,0.017,0.016,0.022
Caltech36-Wisconsin87,0.039,0.069,0.032,0.045,0.088,0.031,0.027,0.037,0.026,0.024,0.020
Simmons81-Reed98,0.035,0.042,0.049,0.031,0.045,0.029,0.027,0.022,0.022,0.023,0.019
Simmons81-NYU9,0.056,0.067,0.058,0.048,0.052,0.068,0.049,0.036,0.020,0.023,0.017
Simmons81-Virginia63,0.037,0.094,0.052,0.039,0.045,0.060,0.053,0.037,0.043,0.035,0.036


In [12]:
# Subgraph sampling for subgraph classification
# Output = subgraph_list as NNetwork objects

ntwk_list = ['Caltech36', 'Simmons81', 'Reed98', 'NYU9', 'Virginia63', 'UCLA26', 'Wisconsin87'] # Wisconsin87, UCLA26, Caltech36

#ntwk_list = ['Caltech36', 'Simmons81', 'Reed98', 'NYU9'] # Wisconsin87, UCLA26, Caltech36

label_list = []
subgraph_list = []
num_subgraphs = 100
k_list = [30]
k0_list = [5,10,15,20]

#X_MACC = []
#X_edge_density = []
#X_adj = []

nn_network_list = []
for ntwk in ntwk_list:
    ntwk_nonumber = ''.join([i for i in ntwk if not i.isdigit()])
    path = "Data/Facebook/" + str(ntwk) + '.txt'
    G = nn.NNetwork()
    G.load_add_edges(path, increment_weights=False, use_genfromtxt=True)
    nn_network_list.append(G)
    print('num nodes in G', len(G.nodes()))
    print('num edges in G', len(G.get_edges()))

y = [0]*num_subgraphs + [1]*num_subgraphs


for trial in np.arange(10):
    results_all = {}
    for a in np.arange(len(nn_network_list)):
        for b in np.arange(a+1, len(nn_network_list)):
            if a != b:
                nn_network_list_sub = [nn_network_list[a],nn_network_list[b]]
                print("Classifying subgraphs in {}-{} ...".format(ntwk_list[a], ntwk_list[b]))

                subgraph_list_dict = {}
                output_dict_list_total = {}

                for k in k_list:
                    print("sampling subgraphs with k={} nodes".format(k))
                    subgraph_list = []
                    for q in np.arange(len(nn_network_list)):
                        G = nn_network_list[q]

                        for i in trange(num_subgraphs):
                            label_list.append(str(ntwk))

                            # take the induced subgraph 
                            #X, embs = G.get_patches(k=k, sample_size=10, skip_folded_hom=False, sampling_alg = 'pivot')
                            #H = G.subgraph(embs[-1]) # take the last instance of MCMC sampling

                            H = G.k_node_ind_subgraph(k=k)
                            while H is None:
                                H = G.k_node_ind_subgraph(k=k)

                            #A_adj = H.get_adjacency_matrix()
                            subgraph_list.append(H)
                            #subgraph_list.append(A_adj)

                            """
                            ## compute summary stats of subgraphs 

                            A_adj = H.get_adjacency_matrix()
                            X_adj.append(A_adj)

                            X0, embs0 = H.get_patches(k=k0, sample_size=1000, skip_folded_hom=False, sampling_alg = 'pivot')
                            MACC = np.sum(X0, axis=1)/X0.shape[1]
                            MACC = MACC.reshape(k0,k0)
                            X_MACC.append(MACC)

                            X_edge_density.append(np.linalg.norm(A_adj, 1)/len(H.nodes()))
                            """

                    subgraph_list_dict.update({"k={}".format(k) : subgraph_list})

                    print("extracting subgraph features..")
                    output_dict_list = datagen_graph_features(X=subgraph_list, k0_list = k0_list)
                    results_dict_total = run_binary_classification(output_dict_list, y=y, scale=0.3)

                    output_dict_list_total.update({"subgraph_list": subgraph_list_dict})
                    output_dict_list_total.update({"k={}".format(k): results_dict_total})
                output_dict_list_total.update({"subgraph_list": subgraph_list_dict})
                results_all.update({"{}-{}".format(ntwk_list[a], ntwk_list[b]) : output_dict_list_total})
                #np.save("Output_files/subgraph_classification_data30_{}".format(int(trial)), results_all)
                np.save("Output_files/subgraph_classification_data30_"+str(trial), results_all)

num nodes in G 769
num edges in G 33312
num nodes in G 1518
num edges in G 65976
num nodes in G 962
num edges in G 37624
num nodes in G 21679
num edges in G 1431430
num nodes in G 21325
num edges in G 1396356
num nodes in G 20467
num edges in G 1495226
num nodes in G 23842
num edges in G 1671904
Classifying subgraphs in Caltech36-Simmons81 ...
sampling subgraphs with k=30 nodes


100%|█████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 76.25it/s]


extracting subgraph features..


100%|█████████████████████████████████████████████████████████████████████| 700/700 [15:37<00:00,  1.34s/it]


X.shape (700, 1)
method = num_nodes, AUC = 0.500000
X.shape (700, 1)
method = num_edges, AUC = 0.913766
X.shape (700, 1)
method = min_degree, AUC = 0.618247
X.shape (700, 1)
method = max_degree, AUC = 0.773709
X.shape (700, 1)
method = diameter, AUC = 0.862945
X.shape (700, 1)
method = degree_assortativity_coef, AUC = 0.638255
X.shape (700, 1)
method = num_cliques, AUC = 0.738295
X.shape (700, 1)
method = Avg_clustering_coeff, AUC = 0.762305
X.shape (700, 1)
method = edeg_density, AUC = 0.913766
X.shape (700, 25)
method = MACC k0=5, AUC = 0.931973
X.shape (700, 100)
method = MACC k0=10, AUC = 0.946779
X.shape (700, 225)
method = MACC k0=15, AUC = 0.965986
X.shape (700, 400)
method = MACC k0=20, AUC = 0.969188
Classifying subgraphs in Caltech36-Reed98 ...
sampling subgraphs with k=30 nodes


...

X.shape (700, 1)
method = num_nodes, AUC = 0.500000
X.shape (700, 1)
method = num_edges, AUC = 0.834334
X.shape (700, 1)
method = min_degree, AUC = 0.593437
X.shape (700, 1)
method = max_degree, AUC = 0.714886
X.shape (700, 1)
method = diameter, AUC = 0.822329
X.shape (700, 1)
method = degree_assortativity_coef, AUC = 0.654662
X.shape (700, 1)
method = num_cliques, AUC = 0.693878
X.shape (700, 1)
method = Avg_clustering_coeff, AUC = 0.710284
X.shape (700, 1)
method = edeg_density, AUC = 0.834334
X.shape (700, 25)
method = MACC k0=5, AUC = 0.911965
X.shape (700, 100)
method = MACC k0=10, AUC = 0.899960
X.shape (700, 225)
method = MACC k0=15, AUC = 0.887955
X.shape (700, 400)
method = MACC k0=20, AUC = 0.916367
Classifying subgraphs in UCLA26-Wisconsin87 ...
sampling subgraphs with k=30 nodes


100%|████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 111.07it/s]


extracting subgraph features..


100%|█████████████████████████████████████████████████████████████████████| 700/700 [15:34<00:00,  1.34s/it]


X.shape (700, 1)
method = num_nodes, AUC = 0.500000
X.shape (700, 1)
method = num_edges, AUC = 0.860744
X.shape (700, 1)
method = min_degree, AUC = 0.649460
X.shape (700, 1)
method = max_degree, AUC = 0.789316
X.shape (700, 1)
method = diameter, AUC = 0.882753
X.shape (700, 1)
method = degree_assortativity_coef, AUC = 0.677471
X.shape (700, 1)
method = num_cliques, AUC = 0.765306
X.shape (700, 1)
method = Avg_clustering_coeff, AUC = 0.789116
X.shape (700, 1)
method = edeg_density, AUC = 0.860744
X.shape (700, 25)
method = MACC k0=5, AUC = 0.910764
X.shape (700, 100)
method = MACC k0=10, AUC = 0.950780
X.shape (700, 225)
method = MACC k0=15, AUC = 0.951581
X.shape (700, 400)
method = MACC k0=20, AUC = 0.963585


In [67]:
path = "Output_files/subgraph_classification_data30.npy"
a = np.load(path, allow_pickle=True).item()

In [70]:
a.keys()

dict_keys(['Wisconsin87-Simmons81', 'Wisconsin87-Reed98', 'Wisconsin87-NYU9', 'Wisconsin87-Virginia63', 'Wisconsin87-UCLA26', 'Wisconsin87-Wisconsin87'])

In [72]:
results_all.keys()

dict_keys(['Wisconsin87-Simmons81', 'Wisconsin87-Reed98', 'Wisconsin87-NYU9', 'Wisconsin87-Virginia63', 'Wisconsin87-UCLA26', 'Wisconsin87-Wisconsin87'])

In [32]:
# Subgraph sampling for subgraph classification
# Output = subgraph_list as NNetwork objects

ntwk_list = ['Wisconsin87', 'UCLA26'] # Wisconsin87, UCLA26, Caltech36
label_list = []
subgraph_list = []
num_subgraphs = 100
k_list = [110]
k0_list = [30,40,50,60,70]

#X_MACC = []
#X_edge_density = []
#X_adj = []

nn_network_list = []
for ntwk in ntwk_list:
    ntwk_nonumber = ''.join([i for i in ntwk if not i.isdigit()])
    path = "Data/Facebook/" + str(ntwk) + '.txt'
    G = nn.NNetwork()
    G.load_add_edges(path, increment_weights=False, use_genfromtxt=True)
    nn_network_list.append(G)
    print('num nodes in G', len(G.nodes()))
    print('num edges in G', len(G.get_edges()))

y = [0]*num_subgraphs + [1]*num_subgraphs

subgraph_list_dict = {}
output_dict_list_total = {}

for k in k_list:
    print("sampling subgraphs with k={} nodes".format(k)
    subgraph_list = []
    for a in np.arange(len(nn_network_list)):
        G = nn_network_list[a]
        
        for i in trange(num_subgraphs):
            label_list.append(str(ntwk))

            # take the induced subgraph 
            X, embs = G.get_patches(k=k, sample_size=10, skip_folded_hom=False, sampling_alg = 'pivot')
            H = G.subgraph(embs[-1]) # take the last instance of MCMC sampling
            #A_adj = H.get_adjacency_matrix()
            subgraph_list.append(H)
            #subgraph_list.append(A_adj)

            """
            ## compute summary stats of subgraphs 

            A_adj = H.get_adjacency_matrix()
            X_adj.append(A_adj)

            X0, embs0 = H.get_patches(k=k0, sample_size=1000, skip_folded_hom=False, sampling_alg = 'pivot')
            MACC = np.sum(X0, axis=1)/X0.shape[1]
            MACC = MACC.reshape(k0,k0)
            X_MACC.append(MACC)

            X_edge_density.append(np.linalg.norm(A_adj, 1)/len(H.nodes()))
            """

    subgraph_list_dict.update({"k={}".format(k) : subgraph_list})

    print("extracting subgraph features..")
    output_dict_list = datagen_graph_features(X=subgraph_list, k0_list = k0_list)
    results_dict_total = run_binary_classification(output_dict_list, y=y, scale=0.3)
    
    output_dict_list_total.update({"k={}".format(k): results_dict_total})

num nodes in G 23842
num edges in G 1671904
num nodes in G 20467
num edges in G 1495226
sampling subgraphs with k=110 nodes


100%|█████████████████████████████████████████| 100/100 [00:19<00:00,  5.20it/s]


extracting subgraph features..


 55%|██████████████████████▌                  | 110/200 [23:43<18:55, 12.61s/it]/Users/hanbaek/opt/miniconda3/envs/hlyuenv/lib/python3.9/site-packages/networkx/algorithms/assortativity/correlation.py:298: RuntimeWarning: invalid value encountered in double_scalars
  return (xy * (M - ab)).sum() / np.sqrt(vara * varb)
100%|█████████████████████████████████████████| 200/200 [45:41<00:00, 13.71s/it]

X.shape (200, 1)
method = num_nodes, AUC = 0.467587
X.shape (200, 1)
method = num_edges, AUC = 0.462185
X.shape (200, 1)
method = min_degree, AUC = 0.513405
X.shape (200, 1)
method = max_degree, AUC = 0.665866
X.shape (200, 1)
method = diameter, AUC = 0.518607
X.shape (200, 1)
method = degree_assortativity_coef, AUC = 0.617047
X.shape (200, 1)
method = num_cliques, AUC = 0.486595
X.shape (200, 1)
method = Avg_clustering_coeff, AUC = 0.656663
X.shape (200, 1)
method = edeg_density, AUC = 0.522209
X.shape (200, 900)
method = MACC k0=30, AUC = 0.745898
X.shape (200, 1600)
method = MACC k0=40, AUC = 0.787915
X.shape (200, 2500)
method = MACC k0=50, AUC = 0.749500
X.shape (200, 3600)
method = MACC k0=60, AUC = 0.757903
X.shape (200, 4900)
method = MACC k0=70, AUC = 0.793517


In [34]:
path = "Output_files/subgraph_classification_ex1"
np.save(path, output_dict_list_total)

In [25]:
# Subgraph sampling for subgraph classification
# Output = subgraph_list as NNetwork objects

ntwk_list = ['UCLA26', 'true_edgelist_for_BA_5000_m_25'] # Wisconsin87, UCLA26, Caltech36
label_list = []
subgraph_list = []
num_subgraphs = 100
k_list = [50]
k0_list = [15,20,30]

#X_MACC = []
#X_edge_density = []
#X_adj = []

nn_network_list = []
for ntwk in ntwk_list:
    ntwk_nonumber = ''.join([i for i in ntwk if not i.isdigit()])
    path = "Data/Facebook/" + str(ntwk) + '.txt'
    G = nn.NNetwork()
    G.load_add_edges(path, increment_weights=False, use_genfromtxt=True)
    nn_network_list.append(G)
    print('num nodes in G', len(G.nodes()))
    print('num edges in G', len(G.get_edges()))

y = [0]*num_subgraphs + [1]*num_subgraphs

subgraph_list_dict = {}
output_dict_list_total = {}

for k in k_list:
    print("sampling subgraphs with k={} nodes".format(k))
    subgraph_list = []
    for a in np.arange(len(nn_network_list)):
        G = nn_network_list[a]
        
        for i in trange(num_subgraphs):
            label_list.append(str(ntwk))

            # take the induced subgraph 
            X, embs = G.get_patches(k=k, sample_size=10, skip_folded_hom=False, sampling_alg = 'pivot')
            H = G.subgraph(embs[-1]) # take the last instance of MCMC sampling
            #A_adj = H.get_adjacency_matrix()
            subgraph_list.append(H)
            #subgraph_list.append(A_adj)

            """
            ## compute summary stats of subgraphs 

            A_adj = H.get_adjacency_matrix()
            X_adj.append(A_adj)

            X0, embs0 = H.get_patches(k=k0, sample_size=1000, skip_folded_hom=False, sampling_alg = 'pivot')
            MACC = np.sum(X0, axis=1)/X0.shape[1]
            MACC = MACC.reshape(k0,k0)
            X_MACC.append(MACC)

            X_edge_density.append(np.linalg.norm(A_adj, 1)/len(H.nodes()))
            """

    subgraph_list_dict.update({"k={}".format(k) : subgraph_list})

    print("extracting subgraph features..")
    output_dict_list = datagen_graph_features(X=subgraph_list, k0_list = k0_list)
    results_dict_total = run_binary_classification(output_dict_list, y=y, scale=0.3)
    
    output_dict_list_total.update({"k={}".format(k): results_dict_total})

num nodes in G 20467
num edges in G 1495226
num nodes in G 5000
num edges in G 248750
sampling subgraphs with k=50 nodes


100%|█████████████████████████████████████████| 100/100 [00:04<00:00, 23.82it/s]


extracting subgraph features..


100%|█████████████████████████████████████████| 200/200 [06:31<00:00,  1.96s/it]

X.shape (200, 1)
method = num_nodes, AUC = 0.602241
X.shape (200, 1)
method = num_edges, AUC = 0.697679
X.shape (200, 1)
method = min_degree, AUC = 0.554222
X.shape (200, 1)
method = max_degree, AUC = 0.846939
X.shape (200, 1)
method = diameter, AUC = 0.925570
X.shape (200, 1)
method = degree_assortativity_coef, AUC = 0.769908
X.shape (200, 1)
method = num_cliques, AUC = 0.736695
X.shape (200, 1)
method = Avg_clustering_coeff, AUC = 0.992397
X.shape (200, 1)
method = edeg_density, AUC = 0.726291
X.shape (200, 225)
method = MACC k0=15, AUC = 0.989596
X.shape (200, 400)
method = MACC k0=20, AUC = 0.993998
X.shape (200, 900)
method = MACC k0=30, AUC = 0.996799


In [ ]:
results_dict_total = run_binary_classification(output_dict_list, scale=1)

In [ ]:
# Using full adjacency matrix 
X = np.asarray(X_adj).reshape(-1,k0**2)
print("X.shape", X.shape)
X_train = X[X_train_idx,:]
X_test = X[X_test_idx,:]

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
P_pred = clf.predict_proba(X_test)
#print("P_pred", P_pred)

compute_accuracy_metrics(y_test, P_pred[:,1], use_opt_threshold=False, verbose=False)

In [ ]:
def display_graphs(title,
                     save_path,
                     grid_shape=[2,3],
                     fig_size=[10,10],
                     data = None, # [X, embs]
                     show_importance=False):

        # columns of X = vectorized k x k adjacency matrices
        # corresponding list in embs = sequence of nodes (may overalp)
        X, embs = data
        print('X.shape', X.shape)

        rows = grid_shape[0]
        cols = grid_shape[1]

        fig = plt.figure(figsize=fig_size, constrained_layout=False)
        # make outer gridspec

        idx = np.arange(X.shape[1])
        outer_grid = gridspec.GridSpec(nrows=rows, ncols=cols, wspace=0.02, hspace=0.05)

        # make nested gridspecs
        for i in range(rows * cols):
            a = i // cols
            b = i % rows

            Ndict_wspace = 0.05
            Ndict_hspace = 0.05

            # display graphs
            inner_grid = outer_grid[i].subgridspec(1, 1, wspace=Ndict_wspace, hspace=Ndict_hspace)

            # get rid of duplicate nodes
            A = X[:,idx[i]]
            A = X[:,idx[i]].reshape(int(np.sqrt(X.shape[0])), -1)
            H = nn.NNetwork()
            H.read_adj(A, embs[idx[i]])
            A_sub = H.get_adjacency_matrix()

            # read in as a nx graph for plotting
            G1 = nx.from_numpy_matrix(A_sub)
            ax = fig.add_subplot(inner_grid[0, 0])
            pos = nx.spring_layout(G1)
            edges = G1.edges()
            weights = [1*G1[u][v]['weight'] for u,v in edges]
            nx.draw(G1, with_labels=False, node_size=20, ax=ax, width=weights, label='Graph')

            ax.set_xticks([])
            ax.set_yticks([])

        plt.suptitle(title, fontsize=15)
        fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0)
        fig.savefig(save_path, bbox_inches='tight')

In [ ]:
def display_dict_and_graph(title,
                           W, 
                           singular_values, 
                         save_path,
                         grid_shape=None,
                         fig_size=[10,10],
                         show_importance=False):
        
        n_components = W.shape[1]
        k = int(np.sqrt(W.shape[0]))

        rows = np.round(np.sqrt(n_components))
        rows = rows.astype(int)
        if grid_shape is not None:
            rows = grid_shape[0]
            cols = grid_shape[1]
        else:
            if rows ** 2 == n_components:
                cols = rows
            else:
                cols = rows + 1

        if show_importance:
            # importance = np.sum(self.code, axis=1) / sum(sum(self.code))
            idx = np.argsort(singular_values)
            idx = np.flip(idx)
        else:
            idx = np.arange(W.shape[1])

        Ndict_wspace = 0.05
        Ndict_hspace = 0.05

        fig = plt.figure(figsize=fig_size, constrained_layout=False)
        outer_grid = gridspec.GridSpec(nrows=1, ncols=2, wspace=0.02, hspace=0.05)
        for t in np.arange(2):
            # make nested gridspecs

            if t == 0:
                ### Make gridspec
                inner_grid = outer_grid[t].subgridspec(rows, cols, wspace=Ndict_wspace, hspace=Ndict_hspace)
                #gs1 = fig.add_gridspec(nrows=rows, ncols=cols, wspace=0.05, hspace=0.05)

                for i in range(rows * cols):
                    a = i // cols
                    b = i % cols
                    ax = fig.add_subplot(inner_grid[a, b])
                    ax.imshow(W.T[idx[i]].reshape(k, k), cmap="viridis", interpolation='nearest')
                    # ax.set_xlabel('%1.2f' % importance[idx[i]], fontsize=13)  # get the largest first
                    # ax.xaxis.set_label_coords(0.5, -0.05)  # adjust location of importance appearing beneath patches
                    ax.set_xticks([])
                    ax.set_yticks([])
            if t == 1:
                inner_grid = outer_grid[t].subgridspec(rows, cols, wspace=Ndict_wspace, hspace=Ndict_hspace)
                #gs1 = fig.add_gridspec(nrows=rows, ncols=cols, wspace=0.05, hspace=0.05)

                for i in range(rows * cols):
                    a = i // cols
                    b = i % cols

                    G1 = nx.from_numpy_matrix(W[:,idx[i]].reshape(int(np.sqrt(W.shape[0])),-1))
                    ax = fig.add_subplot(inner_grid[a, b])
                    pos = nx.spring_layout(G1)
                    edges = G1.edges()
                    weights = [5*G1[u][v]['weight'] for u,v in edges]
                    nx.draw(G1, with_labels=False, node_size=10, ax=ax, width=weights, label='Graph')
                    if show_importance:
                        ax.set_xlabel('%1.2f' % importance[idx[i]], fontsize=13)  # get the largest first
                        ax.xaxis.set_label_coords(0.5, -0.05)  # adjust location of importance appearing beneath patches

                    ax.set_xticks([])
                    ax.set_yticks([])

        plt.suptitle(title, fontsize=25)
        fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0)
        fig.savefig(save_path, bbox_inches='tight')

In [ ]:
sampling_alg = 'pivot'

ntwk = 'Caltech36' # COVID_PPI, Wisconsin87, UCLA26
ntwk_nonumber = ''.join([i for i in ntwk if not i.isdigit()])
save_folder = 'temp/'
k=10

path = "Data/Networks_all_NDL/" + str(ntwk) + '.txt'
G = nn.NNetwork()
G.load_add_edges(path, increment_weights=False, use_genfromtxt=True)
print('num nodes in G', len(G.nodes()))
print('num edges in G', len(G.get_edges()))

#mx0 = G.get_adjacency_matrix(ordered_node_list=G.nodes())
#plt.imshow(mx0)

In [ ]:
X, embs = G.get_patches(k=k, sample_size=1000, skip_folded_hom=True)

display_graphs(title='induced subgraphs on {}-walks in {}'.format(k, ntwk_nonumber),
                 save_path=save_folder + ntwk_nonumber + "_subgraphs_"+ str(sampling_alg) + "_walk", 
                 data = [X, embs],
                 grid_shape = [5, 15],
                 fig_size = [15, 5],
                 show_importance=False)

In [ ]:
np.save("MC_data_matrix", X)

In [ ]:
plt.imshow(X[:,4].reshape(k,k))

In [ ]:
from sklearn.decomposition import PCA ### Use truncated SVD / online PCA later for better computational efficiency 
pca = PCA(n_components=25)
pca.fit(X)

In [ ]:
Y = pca.fit_transform(X)
singular_values = pca.singular_values_

In [ ]:
display_dict_and_graph(title='{}-node induced subgraphs in {} (sampling : {})'.format(k, ntwk_nonumber, sampling_alg),
                 save_path=save_folder + ntwk_nonumber + "_subgraphs_"+ str(sampling_alg), 
                 W = Y,
                 singular_values = singular_values,
                 grid_shape = [5, 5],
                 fig_size = [15, 10],
                 show_importance=False)